In [13]:
import numpy as np
import pandas as pd
import skyfield
from skyfield.api import Loader
skyfield_load = Loader('../data/skyfield')
from skyfield.positionlib import ICRF, Barycentric
from skyfield.toposlib import Topos
import astropy
from astropy.units import deg, au, km, meter, day, minute, second
from astropy.coordinates import SkyCoord, ICRS, GCRS, BarycentricMeanEcliptic, HeliocentricMeanEcliptic, EarthLocation
from scipy.interpolate import CubicSpline
import os
import matplotlib.pyplot as plt

# MSE imports
import astro_utils
from astro_utils import jd_to_mjd, mjd_to_jd
from ra_dec import radec2dir, dir2radec, site2geoloc, qv2dir, direction_diff, radec_diff, skyfield_observe
from horizons_files import load_pos_jpl, load_ast_jpl, load_obs_jpl, load_obs_ast_jpl
from horizons_files import obs_add_interp_qv, obs_ast_add_interp_qv, obs_add_calc_dir, obs_add_radec, obs_direction_diff
import asteroid_integrate
from asteroid_data import make_data_one_file, get_earth_pos
from utils import range_inc

### Observation of Earth and Mars according to JPL

In [14]:
# Data directories
dir_name = '../data/jpl/testing/hourly'
dir_name_daily = '../data/jpl/testing/daily'

# Build DataFrame for earth and mars position at 3 hour frequency
df_earth = load_pos_jpl(body_name='earth', dir_name=dir_name)
df_mars = load_pos_jpl(body_name='mars', dir_name=dir_name)

# Earth at daily frequency
df_earth_daily = load_pos_jpl(body_name='earth', dir_name=dir_name_daily)
# df_mars_daily = load_pos_jpl(body_name='mars', dir_name=dir_name_daily)

### Vectors of First 16 Asteroids from JPL

In [15]:
# Load the asteroid position and velocity from JPL
df_ast = load_ast_jpl(ast_num0=1, ast_num1=16, dir_name=dir_name)
df_ast_daily = load_ast_jpl(ast_num0=1, ast_num1=16, dir_name=dir_name_daily)

### Observations of First 16 Asteroids from JPL

In [16]:
# Load the asteroid observations from JPL
df_obs = load_obs_ast_jpl(ast_num0=1, ast_num1=16, observer_name='palomar', dir_name=dir_name)
df_obs_daily = load_obs_ast_jpl(ast_num0=1, ast_num1=16, observer_name='geocenter', dir_name=dir_name_daily)

### Calculate Asteroid Direction with qv2dir() and MSE Position / Velocity

In [17]:
ast_elt = asteroid_integrate.load_data()
ast_elt.rename(mapper={'Num':'asteroid_num'}, axis='columns', inplace=True)

In [18]:
# Range of asteroids to for data
ast_num_file_start: int = 1
ast_num_file_end: int = 1000
inputs, outputs = make_data_one_file(0, ast_num_file_end)

In [19]:
# The block of asteroid numbers to test (inclusive boundaries)
ast_num_min = 1
ast_num_max = 16

# The number of asteroids, times, and total rows we want to match
# Use daily data to match data file
N_ast = ast_num_max - ast_num_min + 1
N_t = df_earth_daily.mjd.size
N_row = N_ast * N_t
obstime_mjd = df_ast_daily.mjd.values

# Report data shape
print(f'Shape of data frames df_ast and df_obs:')
print(f'N_ast = {N_ast:5} asteroids')
print(f'N_t   = {N_t:5} observation times')
print(f'N_row = {N_row:5} rows in df_ast and df_obs')

Shape of data frames df_ast and df_obs:
N_ast =    16 asteroids
N_t   =  3653 observation times
N_row = 58448 rows in df_ast and df_obs


In [20]:
# Filter for asteroid numbers 
ast_num_file = np.arange(ast_num_file_start, ast_num_file_end, dtype=np.int64)
mask_ast = (ast_num_min <= ast_num_file) & (ast_num_file <= ast_num_max)

# MSE integrated times as one array
ts = inputs['ts'][0]

# Time range for JPL data
t_min = np.min(obstime_mjd)
t_max = np.max(obstime_mjd)

# Filter for MSE times that match
mask_t = (t_min <= ts) & (ts <= t_max)

In [21]:
# Block of asteroid data 
q_ast_all = outputs['q']
v_ast_all = outputs['v']
u_ast_all = outputs['u']

# filter for selected asteroids only
q_ast_all_t = q_ast_all[mask_ast, :, :]
v_ast_all_t = v_ast_all[mask_ast, :, :]
u_ast_all_t = u_ast_all[mask_ast, :, :]

# filter for selected times only
q_ast_mse_3d = q_ast_all_t[:, mask_t, :]
v_ast_mse_3d = v_ast_all_t[:, mask_t, :]
u_ast_mse_3d = u_ast_all_t[:, mask_t, :]

# for some reason i don't understand, can't do these at once
# q_ast_mse = q_ast_all[mask_ast, mask_t, :]
q_ast_mse_3d.shape

(16, 3653, 3)

In [22]:
# Reshape MSE asteroid data to match shape of DataFrame
q_ast_mse = np.zeros((3, N_row))
v_ast_mse = np.zeros((3, N_row))
u_ast_mse = np.zeros((3, N_row))

In [23]:
# Position
q_ast_mse[0, :] = q_ast_mse_3d[:, :, 0].reshape((-1))
q_ast_mse[1, :] = q_ast_mse_3d[:, :, 1].reshape((-1))
q_ast_mse[2, :] = q_ast_mse_3d[:, :, 2].reshape((-1))

# Velocity
v_ast_mse[0, :] = v_ast_mse_3d[:, :, 0].reshape((-1))
v_ast_mse[1, :] = v_ast_mse_3d[:, :, 1].reshape((-1))
v_ast_mse[2, :] = v_ast_mse_3d[:, :, 2].reshape((-1))

# Direction from geocenter
u_ast_mse[0, :] = u_ast_mse_3d[:, :, 0].reshape((-1))
u_ast_mse[1, :] = u_ast_mse_3d[:, :, 1].reshape((-1))
u_ast_mse[2, :] = u_ast_mse_3d[:, :, 2].reshape((-1))

In [24]:
# Compute direction from MSE position and velocity
u_mse = u_ast_mse

# extract u_jpl from df_obs_daily
u_jpl = df_obs_daily[['ux_jpl', 'uy_jpl', 'uz_jpl']].values.T

# difference in directions as a vector
u_diff = u_mse - u_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'mean error: {mean_error:8.3f} arc seconds')

mean error:    0.974 arc seconds
